In [ ]:
lg = function(mu, n, ybar){
    mu2 = mu^2
    n * (ybar *mu - mu2 / 2.0) - log(1 + mu2)
 }

In [ ]:
#metropolis_hstings alg

mh = function(n, ybar, n_iter, mu_init, cand_sd) {
    
    mu_out = numeric(n_iter)
    accpt = 0
    mu_now = mu_init
    lg_now = lg(mu = mu_now, n = n, ybar = ybar)
    
    for (i in 1:n_iter) {
        
        mu_cand = rnorm(1, mean = mu_now, sd = cand_sd)
        
        lg_cand = lg(mu = mu_cand, n = n, ybar = ybar)
        
        lalpha = lg_cand - lg_now
        alpha = exp(lalpha)
        
        
        u = runif(1)
        
        if (u < alpha) { # accept the candidate
            mu_now = mu_cand
            accpt = accpt + 1
            lg_now = lg_cand
        }
        
        mu_out[i] = mu_cand
    }
    
    list(mu=mu_out, accpt=accpt/n_iter)    
}

In [ ]:
y = c(-0.2, -1.5, -5.3, 0.3, -0.8, -2.2)
ybar = mean(y)
n = length(y)

In [ ]:
n = length(y)
hist(y, freq=FALSE, xlim=c(-1.0, 3.0)) # histogram of the data
curve(dt(x=x, df=1), lty=2, add=TRUE) # prior for mu
points(y, rep(0,n), pch=1) # individual data points
points(ybar, 0, pch=19) # sample mean

In [ ]:
set.seed(43) # set the random seed for reproducibility
post = mh(n=n, ybar=ybar, n_iter=5e3, mu_init=0.0, cand_sd=3.0)
str(post)

In [ ]:
ds = c(0.5, 1.5, 3.0, 4.0)

In [ ]:
for (sd in ds) {
    post = mh(n=n, ybar=ybar, n_iter=5e3, mu_init=0.0, cand_sd=sd)
    print(sd)
    print (post$accpt)
}

In [ ]:
post = mh(n=n, ybar=ybar, n_iter=100e3, mu_init=0.0, cand_sd=1.5)

In [ ]:
summary(as.mcmc(post$mu))

In [ ]:
install.packages("coda")
library("coda")

In [ ]:
post = mh(n=n, ybar=ybar, n_iter=1e3, mu_init=0.0, cand_sd=3.0)
str(post)
traceplot(as.mcmc(post$mu))

In [ ]:
post = mh(n=n, ybar=ybar, n_iter=1e3, mu_init=0.0, cand_sd=0.05)
str(post)
traceplot(as.mcmc(post$mu))

In [ ]:
post = mh(n=n, ybar=ybar, n_iter=1e3, mu_init=0.0, cand_sd=0.9)
str(post)
traceplot(as.mcmc(post$mu))

In [ ]:
post = mh(n=n, ybar=ybar, n_iter=1e3, mu_init=30.0, cand_sd=0.9)
post$accpt
traceplot(as.mcmc(post$mu))

In [ ]:
post$mu_keep = post$mu[-c(1:200)] # discard the first 200 samples
plot(density(post$mu_keep, adjust=2.0), main="", xlim=c(-1.0, 3.0), xlab=expression(mu)) # plot density estimate of the posterior
curve(dt(x=x, df=1), lty=2, add=TRUE) # prior for mu
points(ybar, 0, pch=19) # sample mean

curve(0.017*exp(lg(mu=x, n=n, ybar=ybar)), from=-1.0, to=3.0, add=TRUE, col="blue") # approximation to the true posterior in blue